In [1]:
# Fuzzy Gradient Boosting Regression Algorithm
# using different defuzzification methods
# by Resmiye Nasiboglu and Efendi Nasibov
# August, 2022

from sklearn import datasets
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
import seaborn as sns
import numpy as np
import random as r
import pandas as pd
import math
from sklearn import preprocessing
from my_datasets import *
from fuzzy_operations import *

from warnings import filterwarnings
filterwarnings('ignore')
    
def c_cycle(c):
    # c is the optimism parameter of the WABL method.
    print("=============================== c = %.1f" % (c))
    
    """
    ===== DATASETS =====
    my_load_iris()
    my_load_car_prices()
    my_load_diabetes()
    my_load_boston()
    my_load_penguins()
    my_load_planets()
    my_load_diamonds()
    my_load_mpg()
    my_load_tips()
    my_load_taxis()
    """
    
    # replace with the required my_load_...() function
    X_train,X_test,y_train,y_test = my_load_iris()

    r.seed(0) # initialization of the random generator.

    # fuzzy number representation: A=(mode,l_width,r_width)

    # in experiments the folowing forms of the FN are used
    #     l_max_width=0.2 and r_max_witth=0.2  (symmetrical case)
    #     l_max_width=0.2 and r_max_witth=0.0  (left skewned case)
    #     l_max_width=0.0 and r_max_witth=0.2  (right skewned case)
    
    l_max_width=0.2  # left max width of fuzzyness
    r_max_width=0.0  # right max width of fuzzyness

    # reservation of empty fuzzy data 
    y_fuz_train=[[0,0,0] for _ in range(len(y_train))]
    y_fuz_test=[[0,0,0] for _ in range(len(y_test))]
    
    # generation of random fuzzy data 
    for i in range(len(y_train)):
        y_fuz_train[i] = [y_train[i],y_train[i]*(l_max_width*r.random()),y_train[i]*(r_max_width*r.random())]
    for i in range(len(y_test)):    
        y_fuz_test[i] = [y_test[i],y_test[i]*(l_max_width*r.random()),y_test[i]*(r_max_width*r.random())]
    
   
    # Standartization of the inputs

    sc = MinMaxScaler()
    X_train_std = sc.fit_transform(X_train)
    X_test_std = sc.transform(X_test)

    # -------- Parameters ----------
    M=201  # number of boosting iterations
    learning_rate=0.1 
    tree_depth=1  # depth of the stump trees
    max_leaf=2**tree_depth  # maximum leaf number of the stump trees

    F=[[[0,0,0] for i in range(len(X_train))] for j in range(M)]
    
    # fuzzy average of the fuzzy train outputs. c is the optimism parameter of the WABL.
    f_ave=fuzAve(y_fuz_train,c)
  
    # F[i] is the fuzzy outputs of the model after i.th iteration
    F[0]=[f_ave for _ in range(len(X_train))]

    # gamma is the predicted fuzzy output (as a Fuzzy Number) according to the leaf 
    gamma=[[[0,0,0] for i in range(max_leaf)] for j in range(M)]
    trees=[]

    # boosting iterations
    for m in range(1,M):    
        rrr=[fuzSubtr(y_fuz_train[i],F[m-1][i],c) for i in range(len(y_fuz_train))]

        # stump tree is constructed up to the defuzzified values of the FNs
        r1=[defuz(rrr[i],c) for i in range(len(rrr))]

        # constructing of the stump tree
        tree = DecisionTreeRegressor(random_state=0,max_depth=tree_depth)
        tree.fit(X_train_std, r1)
        trees.append(tree)
        
        # h is the list of the indices of the leafs
        h=tree.apply(X_train_std)   

        # h1 is the list of the distinct leaf indices 
        h1=list(set(h))

        for l in range(len(h1)):
            leaf_l=[j for j in range(len(r1)) if h[j]==h1[l]] 
            ss=[rrr[j] for j in leaf_l]
            ss1=np.reshape(ss,(-1,3))
            gamma[m][l]=fuzAve(ss1,c) #for each leaf node
            for k in leaf_l:
                F[m][k]=fuzAdd(F[m-1][k],fuzMultBy(gamma[m][l],learning_rate,c),c) 

    # prediction
    #print("----------- Train set R^2 and fuzRMSE -------------------")
    maxR2=-999999
    minRMSE=-999999
    maxM=-1

    X1=X_train_std
    fuzY=y_fuz_train

    FM=F[0]  
    ave=FM[0]  # average of train set

    for m in range(1,M):
        h=trees[m-1].apply(X1)
        h1=list(set(h))
        for l in range(len(h1)):
            leaf_l=[j for j in range(len(X_train)) if h[j]==h1[l]] 
            for k in leaf_l:
                FF=fuzAdd(FM[k],fuzMultBy(gamma[m][l],learning_rate,c),c)
                FM[k]=FF    #for each xi of each leaf node 

        R2=fuzR2(fuzY,FM,ave,c)
        RMSE=fuzRMSE(fuzY,FM,c)
        if R2>maxR2:
            maxR2=R2
            minRMSE=RMSE
            maxM=m
        if m%10==0:
            pass #print("%3d %10.4f %10.4f"%(m,R2,RMSE))

    print("-- TRAIN: the best R^2 and RMSE values with according iteration number --")
    print("maxM =",maxM)
    print("maxR2 = %10.4f"%(maxR2))
    print("minRMSE = %10.4f"%(minRMSE))

    #print("----------- Test set R^2 and fuzRMSE -------------------")
    maxR2=-999999
    minRMSE=-999999
    maxM=-1

    X1=X_test_std
    fuzY=y_fuz_test
    F=[[[0,0,0] for _ in range(len(X_test))] for i in range(M)]
    
    # initial average of all test set with ave(train)
    F[0]=[fuzAve(y_fuz_train,c) for _ in range(len(X_test))] 
    FM=F[0] 
    #ave=FM[0]  # average of the train set

    for m in range(1,M):
        h=trees[m-1].apply(X1)
        h1=list(set(h))
        for l in range(len(h1)):
            leaf_l=[j for j in range(len(X1)) if h[j]==h1[l]] 
            for k in leaf_l:
                FF=fuzAdd(FM[k],fuzMultBy(gamma[m][l],learning_rate,c),c)
                FM[k]=FF    #for each xi of each leaf node 

        R2=fuzR2(fuzY,FM,ave,c)
        RMSE=fuzRMSE(fuzY,FM,c)
        if R2>maxR2:
            maxR2=R2
            minRMSE=RMSE
            maxM=m
        if m%10==0:
            pass #print("%10.4f %10.4f"%(R2,RMSE))   

    print("-- TEST: the best R^2 and RMSE values with according iteration number --")
    print("maxM =",maxM)
    print("maxR2 = %10.4f"%(maxR2))
    print("minRMSE = %10.4f"%(minRMSE))

# in case of the WABL, calculations are made for all values of the optimism parameter - c
# in case of the other defuzzification methods, this cycle shold be commented
for c in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]:
    c_cycle(c)
    
# in case of the other defuzzification methods, c does not matter. If should be fixed, for example, c = 0.5 
# and the following two lines should be uncommented.
#c=0.5
#c_cycle(c)

=============================== c = 0.0
-- TRAIN: the best R^2 and RMSE values with according iteration number --
maxM = 200
maxR2 =     0.9055
minRMSE =     0.2442
-- TEST: the best R^2 and RMSE values with according iteration number --
maxM = 174
maxR2 =     0.6940
minRMSE =     0.3688
=============================== c = 0.1
-- TRAIN: the best R^2 and RMSE values with according iteration number --
maxM = 200
maxR2 =     0.9166
minRMSE =     0.2281
-- TEST: the best R^2 and RMSE values with according iteration number --
maxM = 174
maxR2 =     0.7132
minRMSE =     0.3559
=============================== c = 0.2
-- TRAIN: the best R^2 and RMSE values with according iteration number --
maxM = 200
maxR2 =     0.9267
minRMSE =     0.2126
-- TEST: the best R^2 and RMSE values with according iteration number --
maxM = 174
maxR2 =     0.7312
minRMSE =     0.3438
=============================== c = 0.3
-- TRAIN: the best R^2 and RMSE values with according iteration number --
maxM = 200
maxR2 = 